In [63]:
import os
import glob
import pandas as pd
import numpy as np
import funcs_for_proc
from transformator import (
    relative_concentration, get_accepted_maximum_value,
    get_all_relative_concentration)
import warnings
warnings.simplefilter("ignore", UserWarning)

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
from itertools import product
from tqdm import tqdm
import tsfel

In [64]:
data = funcs_for_proc.get_data_from_csv('final_data/data_train/data_train', 'csv','class/train.csv')
col=['H2', 'CO', 'C2H4', 'C2H2','H2v', 'COv', 'C2H4v', 'C2H2v','H2a', 'COa', 'C2H4a', 'C2H2a']

In [65]:
data_with_egde_val=pd.DataFrame(columns=col+['status'])

data_for_an=data
cfg = tsfel.get_features_by_domain()
pbar = tqdm(total=len(data_for_an))
for i,file in enumerate(data_for_an):
    insertdata=[]
    datav=file[2].diff()
    ex_f_df = tsfel.time_series_features_extractor(cfg, file[2],verbose=0)
    if i==0:
        data_with_egde_val_ef=pd.DataFrame(columns=ex_f_df.columns)
    data_with_egde_val_ef=data_with_egde_val_ef.append(ex_f_df,ignore_index=True)
    dataa=datav.diff()
    vmax_last=list(datav.iloc[len(datav)-1].values)
    amax_last=list(dataa.iloc[len(dataa)-1].values)
    con=list(file[2].iloc[len(file[2])-1].values)
    status=[file[1]]
    insertdata=con+vmax_last+amax_last+status
    data_with_egde_val.loc[i]=insertdata
    pbar.update(1)
pbar.close()
data_with_egde_val=pd.concat([data_with_egde_val, data_with_egde_val_ef], axis=1)
    

100%|██████████| 2100/2100 [02:28<00:00, 14.18it/s]


In [66]:
data_with_egde_val_ef.head()

,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF Slope,0_ECDF_0,...,3_Wavelet variance_0,3_Wavelet variance_1,3_Wavelet variance_2,3_Wavelet variance_3,3_Wavelet variance_4,3_Wavelet variance_5,3_Wavelet variance_6,3_Wavelet variance_7,3_Wavelet variance_8,3_Zero crossing rate
0,0.001106,0.006647,0.001106,2.597094,84.0,336.0,0.001216,0.001925,743.998763,0.002381,...,1.638706e-10,5.893392e-10,1.291968e-09,2.264273e-09,3.499456e-09,4.990925e-09,6.732171e-09,8.716697e-09,1.093794e-08,0.0
1,0.002056,0.009233,0.002056,2.288400,84.0,336.0,0.002099,0.002336,2077.745712,0.002381,...,2.339277e-11,8.472077e-11,1.870050e-10,3.300260e-10,5.136521e-10,7.377403e-10,1.002095e-09,1.306444e-09,1.650421e-09,0.0
2,0.000563,0.004748,0.000563,2.548840,84.0,336.0,0.000976,0.001327,1029.689330,0.002381,...,6.100834e-11,2.220033e-10,4.922065e-10,8.724728e-10,1.363959e-09,1.967899e-09,2.685527e-09,3.518034e-09,4.466538e-09,0.0
3,0.000416,0.003989,0.000416,2.783991,84.0,336.0,0.000698,0.001238,755.554399,0.002381,...,1.539308e-10,5.523105e-10,1.207931e-09,2.111605e-09,3.254478e-09,4.627566e-09,6.221650e-09,8.027252e-09,1.003461e-08,0.0
4,0.001572,0.008099,0.001572,2.121009,84.0,336.0,0.001855,0.002015,3532.979908,0.002381,...,1.592310e-10,5.733399e-10,1.258149e-09,2.206824e-09,3.412843e-09,4.869495e-09,6.569796e-09,8.506477e-09,1.067203e-08,0.0


In [67]:
data_with_egde_val

,H2,CO,C2H4,C2H2,H2v,COv,C2H4v,C2H2v,H2a,COa,...,3_Wavelet variance_0,3_Wavelet variance_1,3_Wavelet variance_2,3_Wavelet variance_3,3_Wavelet variance_4,3_Wavelet variance_5,3_Wavelet variance_6,3_Wavelet variance_7,3_Wavelet variance_8,3_Zero crossing rate
0,0.002294,0.042099,0.004630,0.000345,0.000004,0.000075,0.000020,5.818826e-07,-2.304700e-09,1.774780e-07,...,1.638706e-10,5.893392e-10,1.291968e-09,2.264273e-09,3.499456e-09,4.990925e-09,6.732171e-09,8.716697e-09,1.093794e-08,0.0
1,0.002764,0.037128,0.006520,0.000146,0.000005,0.000054,0.000033,6.528160e-07,-4.561590e-08,-4.215120e-07,...,2.339277e-11,8.472077e-11,1.870050e-10,3.300260e-10,5.136521e-10,7.377403e-10,1.002095e-09,1.306444e-09,1.650421e-09,0.0
2,0.001816,0.036885,0.002999,0.000253,0.000006,0.000087,0.000022,1.189219e-06,-1.104300e-08,-3.439780e-07,...,6.100834e-11,2.220033e-10,4.922065e-10,8.724728e-10,1.363959e-09,1.967899e-09,2.685527e-09,3.518034e-09,4.466538e-09,0.0
3,0.001659,0.028044,0.008604,0.000310,0.000006,0.000062,0.000019,1.690292e-07,-3.560000e-09,-4.741500e-08,...,1.539308e-10,5.523105e-10,1.207931e-09,2.111605e-09,3.254478e-09,4.627566e-09,6.221650e-09,8.027252e-09,1.003461e-08,0.0
4,0.002091,0.014124,0.013251,0.000347,0.000005,0.000044,0.000029,8.600990e-07,5.168470e-08,2.986550e-07,...,1.592310e-10,5.733399e-10,1.258149e-09,2.206824e-09,3.412843e-09,4.869495e-09,6.569796e-09,8.506477e-09,1.067203e-08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.002149,0.011889,0.006705,0.000209,0.000007,0.000021,0.000026,6.132419e-07,3.950630e-08,3.650550e-07,...,4.873821e-11,1.758848e-10,3.867101e-10,6.795066e-10,1.052630e-09,1.504377e-09,2.032981e-09,2.636621e-09,3.313439e-09,0.0
2096,0.001105,0.035809,0.007522,0.000368,0.000002,0.000079,0.000026,1.244349e-06,-3.644930e-08,-5.906600e-07,...,1.333880e-10,4.832563e-10,1.067433e-09,1.885943e-09,2.940184e-09,4.232495e-09,5.765983e-09,7.544359e-09,9.571750e-09,0.0
2097,0.004339,0.031754,0.004703,0.000291,0.000012,0.000109,0.000022,3.293170e-07,1.763760e-08,5.331860e-07,...,1.407091e-10,5.049864e-10,1.104562e-09,1.930872e-09,2.975390e-09,4.229178e-09,5.682811e-09,7.326404e-09,9.149690e-09,0.0
2098,0.003762,0.022104,0.004537,0.000454,0.000006,0.000070,0.000024,1.327107e-06,-2.584230e-08,-3.851510e-07,...,2.585769e-10,9.323517e-10,2.049322e-09,3.601641e-09,5.582952e-09,7.987609e-09,1.081047e-08,1.404680e-08,1.769211e-08,0.0


In [68]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

In [69]:
data_full = data_with_egde_val.copy()
X_data = data_full.drop('status', axis=1)
y = data_full.status

In [70]:
seed = 7
test_size = 0.3
X_trian, X_test, y_train, y_test = train_test_split(X_data, y, test_size=test_size, random_state=seed)

In [ ]:

param_test = {
 'max_depth':[2,3,4,5,6,7,8,10,12,20,50,100,200],
 'min_child_weight':[2,3,4,5,6,7,8,50,100],
  'gamma':np.linspace(0, 1, num = 10),
    'subsample':np.linspace(0, 1, num = 10),
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, colsample_bytree=0.8,        
  nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring=f1,n_jobs=24,iid=False, cv=5,verbose=8)

model = gsearch.fit(X_trian, y_train)
pred = model.predict(X_test)
print("Accuracy for model: %.2f" % (f1_score(y_test, pred,average='macro')))

Fitting 5 folds for each of 11700 candidates, totalling 58500 fits


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done  24 tasks      | elapsed:   36.6s
[Parallel(n_jobs=24)]: Done  65 tasks      | elapsed:  1.6min
[Parallel(n_jobs=24)]: Done 114 tasks      | elapsed:  2.7min
[Parallel(n_jobs=24)]: Done 173 tasks      | elapsed:  3.9min
[Parallel(n_jobs=24)]: Done 240 tasks      | elapsed:  5.3min
[Parallel(n_jobs=24)]: Done 317 tasks      | elapsed:  6.6min
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:  7.8min
[Parallel(n_jobs=24)]: Done 497 tasks      | elapsed:  9.5min
[Parallel(n_jobs=24)]: Done 600 tasks      | elapsed: 12.1min
[Parallel(n_jobs=24)]: Done 713 tasks      | elapsed: 14.7min
[Parallel(n_jobs=24)]: Done 834 tasks      | elapsed: 17.0min
[Parallel(n_jobs=24)]: Done 965 tasks      | elapsed: 19.5min
[Parallel(n_jobs=24)]: Done 1104 tasks      | elapsed: 23.1min
[Parallel(n_jobs=24)]: Done 1253 tasks      | elapsed: 26.4min
[Parallel(n_jobs=24)]: Done 1410 tasks      | elapsed

EVO

In [ ]:
data_evo = funcs_for_proc.get_data_from_csv('class/data_test/data_test', 'csv',False)

In [ ]:
data_with_egde_val_evo=pd.DataFrame(columns=['filename']+col)
data_for_an_evo=data_evo
cfg = tsfel.get_features_by_domain()

for i,file in enumerate(data_for_an_evo):
    insertdata=[]
    datav=file[2].diff()
    dataa=datav.diff()
    ex_f_df = tsfel.time_series_features_extractor(cfg, file[2],verbose=0)
    if i==0:
        data_with_egde_val_ef_evo=pd.DataFrame(columns=ex_f_df.columns)
    data_with_egde_val_ef_evo=data_with_egde_val_ef_evo.append(ex_f_df,ignore_index=True)
    vmax_last=list(datav.iloc[len(datav)-1].values)
    amax_last=list(dataa.iloc[len(dataa)-1].values)
    vmax_last=list(datav.iloc[len(datav)-1].values)
    con=list(file[2].iloc[len(file[2])-1].values)
    insertdata=[file[0]]+con+vmax_last+amax_last
    data_with_egde_val_evo.loc[i]=insertdata
data_with_egde_val_evo=pd.concat([data_with_egde_val_evo, data_with_egde_val_ef_evo], axis=1)

    

In [ ]:
data_with_egde_val_evo.head()

In [ ]:
data_full_evo = data_with_egde_val_evo.copy()
X_data_test_evo = data_full_evo.drop('filename', axis=1)


In [ ]:
pred_evo = model.predict(X_data_test_evo)

In [ ]:
exportdata=pd.DataFrame([data_full_evo.filename,pred_evo.astype(int)]).T

In [ ]:
exportdata.head()

In [ ]:
exportdata.to_csv('results_evo/xgb_mod_v4.csv',header=['id','category'],index=False)

In [53]:
import joblib

In [55]:
joblib.dump(gsearch, "xgb1.joblib.dat")

['xgb1.joblib.dat']

In [56]:
z